# Scrape NYT

In [ ]:
# Jupyter magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# imports
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import pandas as pd
import pathlib
from seleniumrequests import Chrome
import time

In [ ]:
save_dir = "../1_data/NYT_corner_office"
pathlib.Path(save_dir).exists()

## Create index of URLs for journal posts

In [ ]:
url = "https://www.nytimes.com/column/corner-office"


# Change this to where the chromedriver is in your environment
path_to_chromedriver = '../chromedriver'

driver = Chrome(executable_path=path_to_chromedriver)
driver.get(url)

In [ ]:
all_links = []
last_article_count = 0

In [ ]:
def filter_out_articles(all_links):
    section_urls = []
    other = []
    good_links = []

    for l in all_links:
        if l is None:
            continue
        if 'section' in l:
            section_urls.append(l)
            continue
        if 'www.nytimes' not in l:
            other.append(l)
            continue
        if 'subscription' in l:
            continue
        if 'action=click' in l:
            continue
        if '/privacy/' in l:
            continue
        if '/reader-center/' in l:
            continue
        if '/marketing/' in l:
            continue
            
        n = len(l.split('https://www.nytimes.com/')[-1].split('/'))
        if n != 5:
#             print(n, l)
            continue

        else:        
            good_links.append(l)
            
    return good_links


els = driver.find_elements_by_tag_name('a')
new_links = [l.get_attribute('href') for l in els]
all_links += new_links
all_links = list(set(all_links))
old_count = len(article_links)
article_links = filter_out_articles(all_links)
new_count = len(article_links)
print(f"{len(new_links)} new links, {len(all_links)} total links, {len(set(article_links))} articles")

if new_count > old_count:
    print('Got {new_count - old_count}!')

In [ ]:
with open(f"{save_dir}/index.txt", "w") as f:
    f.write("\n".join(article_links))

In [ ]:
with open(f"{save_dir}/index.txt", "r") as f:
    print(f.read())

# Start going through the index

In [ ]:
url = article_links[0]
url

## Grab the text for each post

Some manual fiddling of Selenium required

In [ ]:
all_titles = []
all_text = []

In [ ]:
junk = ['ADVERTISEMENT', "", 'Offer extended: Subscribe for $1 a week.']
def get_article(driver):

    title = driver.find_element_by_tag_name('h1').text
    text = [e.text for e in driver.find_elements_by_tag_name('p')]
    text = [t for t in text if t not in junk]
    
    for j in junk:
        try:
            text.remove(j)
        except:
            continue
    
    return title, text


def process_url(driver, url):
    # go to article
    driver.get(url)
    # time.sleep(random.random() * 3)

    time.sleep(2 + random.random() * 2)
    try:
        buttons = driver.find_elements_by_tag_name('button')
        for b in buttons:
            if "Show Full Article" in b.text:
                print(b.text)
                b.click()
    except:
        pass


    title, text = get_article(driver)
    all_titles.append(title)
    all_text.append(text)
    print(len(text), title)

    with open(f"{save_dir}/{title}.txt", "w") as f:
        f.write("\n".join(text))
    return title, text
        


In [ ]:
start = 102
for i, url in enumerate(article_links):
    if i < start:
        continue
    print(i, url)
    title, text = process_url(driver, url)
    time.sleep(4 + random.random() * 4)
    if i % 5 == 0:
        time.sleep(30 + random.random() * 10)



In [ ]:
text_bit = []
new_text = []
for a in all_text:
    if len(a) < 9:
        continue
    b = "".join(a[1:10])
    if b in text_bit:
        continue
    text_bit.append(b)
    new_text.append(a)
len(new_text)

In [ ]:
df_index = pd.DataFrame([all_titles, article_links], index=['title', 'url']).T

# Manual Inensive data munging to pull out names

In [ ]:
def parse_setup(setup):

#     print('interview with' in setup, setup)
    if 'interview with' in setup:
        name = setup.split('This interview with')[1].split(',')[0]
#     elif 'interview of' in setup:
#         name = setup.split('This interview of')[1].split(',')[0]
    job_title = setup.split(',')[1]
    return name, job_title

def find_setup(txt):
    setup = ""
    for t in txt:
        if "This interview with" in t:
            return t
#         if "This interview of" in t:
#             return t

alternate_txt = [] 
alternate_titles = []
data = []
for title, txt, url in zip(all_titles, new_text, article_links):
    
    
    name, job_title = "na", ""
    setup = find_setup(txt)
    if setup is not None:
        try:
            name, job_title = parse_setup(setup)
        except:
            print('WARNING')
            print(setup)
    else:
        alternate_txt.append(txt)
        alternate_titles.append(title)
    fname = 'check'
    
    d = {
        'title': title,
        'url': url,
        'year':year,
        'filename': fname,
        'name': name,
        'job_title': job_title,
    }
    data.append(d)
#     print(txt[:3])
#     print(url)
#     year = url.split('.com/')[1].split('/')[0]
#     print(f'{year}----------{setup_found}')


In [ ]:
d = pd.DataFrame(data)
len(d[d['name'] == 'na'])

In [ ]:
n =['Arne Sorenson', 'Michael Evans', 'Keith Mestrich', 
    'Paul Polman', 'Jeff Raider', 'multiple', 'Bernardo Hees', 'Jo Ann Jenkins', 
    "Stacy Brown-Philpot",
   'Claire Babineaux-Fontenot',
    'Maigread Eichten',
    'Matt Mullenweg',
    'Chuck Robbins',
    'Indra Nooyi',
   'Sara Horowitz', #just lastname find
   'Sundar Pichai',
    'Steven Corwin',
    'Marc Benioff',
    'Lonnie Bunch',
    'Katrina Lake',
    'Julie Sweet',
    'Mellody Hobson',
    'Adena Friedman',
    'Gail McGovern',
    'Thasunda Brown Duckett',
    'Stephen Ross',
    'John Donahoe',
    'James Dyson',
    'Hadi Partovi',
    'Larry Praeger',
    'Eileen Fisher',
    'Richard Anderson',
    'Al Kelly',
    'Guy Kawasaki',
    'Ken Frazier',
    'Alexis Ohanian',
    'Michael Mathieu',
    'Emily Powell',
    'Doug Parker',
    'Michael Dell',
    'Joey Bergstein',
    'Michele Roberts',
    'Rebecca Lynn',
    'Mark Hoplamazian',
    'Julia Hartz',
    'Ajay Banga',
    'Marissa Mayer',
    'Dan Rosensweig',
    'Chris Paul',
    'Vas Narasimhan',
    'Darren Walker',
    'Patricia de Stacy Harrison',
    'Niki Leondakis',
    'Mark Bertolini',
    'Mary Kay Henry',
    'Gwyneth Paltrow',
    'Payal Kadakia',
    'Eduardo Castro-Wright',
    'Daniel P. Amos',
    'Jane Goodall',
    'Leana Wen',
    'John Chambers',
    'Sarah Friar',
    'Jacqueline Kosecoff',
    'Laurene Powell Jobs',
    'Gregg Renfrew',
    'Adam Bryant',
    'Hamdi Ulukaya',
    'Roland Swenson',
    'Linda Findley Kozlowski',
    'Tristan Walker',
    'Dan Schulman',
    'Will Wright',
    'Michele Buck',
    'Clarence Otis Jr.',
    'Richard Branson and Holly Branson',
    'Susan Lyne',
    'Reid Hoffman',
    'David Miliband',
    'Steven Ballmer',
    'Steve Mollenkopf',
    'John Mackey',
    'Paula Schneider',
    'David Rubenstein',
    'Dany Levy',
    'Lynn Jurich',
    'Maverick Carter'
   ]

# count = 0
# for i, (title, txt) in enumerate(zip(alternate_titles, alternate_txt)):
#     print('----')

#     if i < len(n):
#         print(n[i])
# #         continue

#     print(txt[:3])
#     for t in txt:
#         if 'interview' in t:
#             print(t)
#             break
#         if 'chief' in t:
#             print(t)
#             break
#         if 'grew up' in t:
#             print(t)
#             break
#     else:
#         print(txt[:3])
            
#     print(title)
#     count += 1
    

len(d[d['name'] == 'na']), len(n)

In [ ]:
# d.loc[d['name'] == '', 'name'] = 'John Chambers'


In [ ]:
fnames = []
for i, row in d.iterrows():
    n = row['name']
    last_name = n.split()[-1]
#     print(n, '-', last_name)
    fname = f"{last_name.lower()}_nyt_{year}"
    fnames.append(fname)

d['filename'] = fnames

In [ ]:
d.to_csv(f"{save_dir}/index.csv")

In [ ]:
for i, row in d.iterrows():
    txt = [t for t in new_text[i] if t not in ['CORNER OFFICE', "", "by Adam Bryant"]]
    with open(f"{save_dir}/{row['filename']}.txt", "w") as f:
        f.write("\n".join(txt))

In [ ]:
row['filename']

In [ ]:
# pd.read_csv(f"{save_dir}/index.csv", index_col=0)